In [2]:
from pandas import read_csv as read
import pandas as pd
import random

import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

from plot import pltPRcurve, pltROCcurve

In [3]:
def plot_corr():
    correlation_matrix = data.corr()
    plt.figure(figsize=(10,8))
    ax = sns.heatmap(correlation_matrix, vmax=1, square=True, annot=True,fmt='.2f', cmap ='GnBu', cbar_kws={"shrink": .5}, robust=True)
    plt.title('Correlation matrix between the features', fontsize=20)
    plt.savefig("cor.png")
    plt.show()

In [4]:
def modExec():
    model.fit(X_train, y_train)
    expected = y_test
    predicted = model.predict(X_test)

    pobas = model.predict_proba(X_test)
    preds = pobas[:, 1]

    report = metrics.classification_report(expected, predicted)
    conf_matrix = metrics.confusion_matrix(expected, predicted)

    print(type(model).__name__)
    print(f"Acc: {accuracy_score(expected, predicted)}")
    print(f"Pre: {precision_score(expected, predicted)}")
    print(f"Re: {recall_score(expected, predicted)}")
    print(f"F1: {f1_score(expected, predicted)}")
    
    print(conf_matrix)

In [5]:
path = "filtData500k.csv"
data = read(path, delimiter=",")

In [6]:
data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])

In [7]:
X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

In [8]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
modExec()

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
modExec()

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()

GaussianNB
Acc: 0.964697212709697
Pre: 0.595460614152203
Re: 0.20468104635153742
F1: 0.3046448087431694
[[110381    606]
 [  3466    892]]
KNeighborsClassifier
Acc: 0.9908188478044129
Pre: 0.9545329291815927
Re: 0.7948600275355667
F1: 0.8674095405033178
[[110822    165]
 [   894   3464]]
RandomForestClassifier
Acc: 0.992838874680307
Pre: 0.9960674157303371
Re: 0.8136759981642956
F1: 0.8956807274564285
[[110973     14]
 [   812   3546]]


In [ ]:
plot_corr()

In [ ]:
data.head()

In [ ]:
data.drop('newbalanceOrig', axis=1, inplace=True)
data.drop('newbalanceDest', axis=1, inplace=True)
data.drop('oldbalanceDest', axis=1, inplace=True)


plot_corr()

In [ ]:
path = "newFilt500k.csv"
data = read(path, delimiter=",")
data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)
data.drop('newbalanceOrig', axis=1, inplace=True)
data.drop('newbalanceDest', axis=1, inplace=True)
data.drop('oldbalanceDest', axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
plot_corr()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])

X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

In [ ]:
from sklearn.feature_selection import RFE

# feature extraction

model = GaussianNB()

rfe = RFE(model, 5)

fit = rfe.fit(X, y)

print(fit.support_)
print(fit.ranking_)

In [ ]:
from sklearn.feature_selection import RFE

# feature extraction

model = KNeighborsClassifier(n_neighbors=3)

rfe = RFE(model, 5)

fit = rfe.fit(X, y)

print(fit.support_)
print(fit.ranking_)

In [ ]:
from sklearn.feature_selection import RFE

# feature extraction

model = RandomForestClassifier(n_estimators=110, n_jobs=-1)

rfe = RFE(model, 5)

fit = rfe.fit(X, y)

print(fit.support_)
print(fit.ranking_)

In [ ]:
model.fit(X_train, y_train)
print(model.feature_importances_)

In [ ]:
l = model.feature_importances_.tolist()
print(l)
l.sort()



In [ ]:
data.head()

In [ ]:
data.drop('newSender', axis=1, inplace=True)
data.drop('fraudsEarly', axis=1, inplace=True)
data.drop('LTS', axis=1, inplace=True)
data.drop('newReceiver', axis=1, inplace=True)
data.head()

In [ ]:

model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()

In [ ]:
data.drop('merchant', axis=1, inplace=True)
data.drop('LTR', axis=1, inplace=True)

data.head()

In [ ]:
data.drop('step', axis=1, inplace=True)
data.head()

In [ ]:
model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()

In [ ]:
data.drop('hour', axis=1, inplace=True)
data.head()

In [ ]:
model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()

In [9]:
path = "newFilt500k.csv"
data = read(path, delimiter=",")
data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)
data.drop('newbalanceOrig', axis=1, inplace=True)
data.drop('newbalanceDest', axis=1, inplace=True)
data.drop('newSender', axis=1, inplace=True)
data.drop('fraudsEarly', axis=1, inplace=True)
data.drop('LTS', axis=1, inplace=True)
data.drop('newReceiver', axis=1, inplace=True)
data.drop('step', axis=1, inplace=True)
data.drop('merchant', axis=1, inplace=True)
data.drop('LTR', axis=1, inplace=True)
data.drop('oldbalanceDest', axis=1, inplace=True)


data.head()

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])

X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

from sklearn import preprocessing
# normalize the data attributes
normalized_X = preprocessing.normalize(X)
# standardize the data attributes
standardized_X = preprocessing.scale(X)

model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()
data.head()

GaussianNB
Acc: 0.9634661233690234
Pre: 0.547682119205298
Re: 0.18976594768242314
F1: 0.28186775732788
[[110304    683]
 [  3531    827]]
KNeighborsClassifier
Acc: 0.9967575534266765
Pre: 0.9556267154620312
Re: 0.9586966498393759
F1: 0.9571592210767469
[[110793    194]
 [   180   4178]]
RandomForestClassifier
Acc: 0.9962460444752699
Pre: 0.9854068760821172
Re: 0.914180816888481
F1: 0.9484585168432329
[[110928     59]
 [   374   3984]]


,type,amount,oldbalanceOrg,isFraud,hour,IZoB
0,4,181.00,181.0,1,1,1
1,1,181.00,181.0,1,1,1
2,3,7107.77,183195.0,0,1,0
3,3,671.64,15123.0,0,1,0
4,3,1373.43,13854.0,0,1,0


In [ ]:
import random

models = []
models.append(RandomForestClassifier(n_estimators=110, n_jobs=-1))
models.append(KNeighborsClassifier(n_neighbors=3))
models.append(GaussianNB())

for model in models:
    
    model.fit(X_train, y_train)
    expected = y_test
    predicted = model.predict(X_test)
    
    pobas = model.predict_proba(X_test)
    preds = pobas[:, 1]
    
    name = type(model).__name__
    
    precision, recall, thresholds = precision_recall_curve(expected, preds)
    # calculate F1 score
    f1 = f1_score(expected, predicted)
    # calculate precision-recall AUC
    au = auc(recall, precision)
    # calculate average precision score
    ap = average_precision_score(expected, preds)
    s = 'f1=%.3f auc=%.3f' % (f1, au)
    print(s)
    # plot the precision-recall curve for the model
    plt.title("Precision-recall curve")

    plt.plot(recall, precision, 'b', label=f'PR {name}\n{s}',  color=(random.uniform(0,1), random.uniform(0, 1), random.uniform(0, 1)))
    plt.legend(loc = 'lower left')

    plt.xlabel('Recall')
    plt.ylabel('Precision')


plt.savefig("PR.png")

In [11]:
X = normalized_X
X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()
data.head()

GaussianNB
Acc: 0.9615414625688153
Pre: 0.1953125
Re: 0.005736576411197797
F1: 0.011145786892554615
[[110884    103]
 [  4333     25]]
KNeighborsClassifier
Acc: 0.9989596428106984
Pre: 0.9976514795678723
Re: 0.9747590637907297
F1: 0.9860724233983287
[[110977     10]
 [   110   4248]]
RandomForestClassifier
Acc: 0.9998005982053838
Pre: 0.9997694258704174
Re: 0.9949518127581459
F1: 0.9973548016101208
[[110986      1]
 [    22   4336]]


,type,amount,oldbalanceOrg,isFraud,hour,IZoB
0,4,181.00,181.0,1,1,1
1,1,181.00,181.0,1,1,1
2,3,7107.77,183195.0,0,1,0
3,3,671.64,15123.0,0,1,0
4,3,1373.43,13854.0,0,1,0


In [12]:
X = standardized_X
X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()
data.head()

GaussianNB
Acc: 0.9601976678659673
Pre: 0.4589644240929905
Re: 0.2989903625516292
F1: 0.3620953174934
[[109451   1536]
 [  3055   1303]]
KNeighborsClassifier
Acc: 0.9910702674584941
Pre: 0.956140350877193
Re: 0.8003671408903167
F1: 0.8713464901324007
[[110827    160]
 [   870   3488]]
RandomForestClassifier
Acc: 0.9962026962590489
Pre: 0.9861111111111112
Re: 0.9123451124368976
F1: 0.9477949940405244
[[110931     56]
 [   382   3976]]


,type,amount,oldbalanceOrg,isFraud,hour,IZoB
0,4,181.00,181.0,1,1,1
1,1,181.00,181.0,1,1,1
2,3,7107.77,183195.0,0,1,0
3,3,671.64,15123.0,0,1,0
4,3,1373.43,13854.0,0,1,0


In [55]:
path = "newFilt500k.csv"
data = read(path, delimiter=",")
data.drop('isFlaggedFraud', axis=1, inplace=True)
data.drop('nameOrig', axis=1, inplace=True)
data.drop('nameDest', axis=1, inplace=True)
data.drop('newbalanceOrig', axis=1, inplace=True)
data.drop('newbalanceDest', axis=1, inplace=True)
data.drop('newSender', axis=1, inplace=True)
data.drop('fraudsEarly', axis=1, inplace=True)
data.drop('LTS', axis=1, inplace=True)
data.drop('newReceiver', axis=1, inplace=True)
data.drop('step', axis=1, inplace=True)
data.drop('merchant', axis=1, inplace=True)
data.drop('LTR', axis=1, inplace=True)
data.drop('oldbalanceDest', axis=1, inplace=True)

In [56]:
data['amountToOld'] = data['amount'] / data['oldbalanceOrg']

data.loc[data.amountToOld > 1000000000, 'amountToOld'] = -1
data.fillna(-1)
data.drop('amount', axis=1, inplace=True)
data.drop('oldbalanceOrg', axis=1, inplace=True)
data['amountToOld'].max()

438304.595

In [57]:
le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])

X = data.loc[:, data.columns != 'isFraud'].values
y = data.loc[:, 'isFraud'].values

y=y.astype('int') # не совсем понял, почему https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown

from sklearn.model_selection import train_test_split as train

X_train, X_test, y_train, y_test = train(X, y, shuffle=False)

model = GaussianNB()
modExec()


model = KNeighborsClassifier(n_neighbors=3)
modExec()


model = RandomForestClassifier(n_estimators=110, n_jobs=-1)
modExec()
data.head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').